In [ ]:
import numpy as np
import tifffile as tiff
import random
import scipy.ndimage
import os
from tqdm import tqdm

# Set deterministic seed
seed = 42
random.seed(seed)
np.random.seed(seed)

# Load a stack of TIFF images from a directory
def load_tiff_stack_from_dir(directory):
    '''Loads a stack of TIFF images from a directory into a 3D NumPy array.'''
    tiff_files = sorted([f for f in os.listdir(directory) if f.lower().endswith('.tiff') or f.lower().endswith('.tif')])
    
    if not tiff_files:
        raise FileNotFoundError('No TIFF files found in the directory.')

    stack = [tiff.imread(os.path.join(directory, f)) for f in tiff_files]
    return np.stack(stack, axis=0)  # Stack images along the first dimension (depth)

# Save dataset to a directory
def save_tiff_stack_to_dir(volume, output_dir):
    '''Saves a 3D NumPy array as individual TIFF images in a directory.'''
    os.makedirs(output_dir, exist_ok=True)
    
    for i in range(volume.shape[0]):
        tiff.imwrite(os.path.join(output_dir, f'slice_{i:03d}.tiff'), volume[i])

# Custom Flip and Rotate Transformations
def mirror_3d(volume, mode):
    """Applies horizontal, vertical, or both mirroring to a 3D volume.
    
    Args:
        volume (np.ndarray): 3D volume to be mirrored.
        mode (str): "horizontal", "vertical", or "both" (default: "both").
    
    Returns:
        np.ndarray: Mirrored 3D volume.
    """
    if mode == "horizontal":
        volume = np.flip(volume, axis=2)  # Flip along width (X-axis)
    elif mode == "vertical":
        volume = np.flip(volume, axis=1)  # Flip along height (Y-axis)
    elif mode == "both":
        volume = np.flip(volume, axis=1)  # Flip along height (Y-axis)
        volume = np.flip(volume, axis=2)  # Flip along width (X-axis)
    else:
        raise ValueError("Mode must be 'horizontal', 'vertical', or 'both'.")

    return volume


def rotate_90_3d(volume, num_rot):
    '''Rotates a 3D volume by 90 degrees along the last two dimensions.'''
    return np.rot90(volume, k=num_rot, axes=(1, 2))  # Rotate in the (height, width) plane


# Shearing Crop (removes fixed % of height, width, or both from both sides)
def shear_crop_3d(volume, crop_percent=0.3, crop_dim='height'):
    '''Shears off a fixed percentage of height, width, or both from both sides of a 3D volume.'''
    d, h, w = volume.shape

    if crop_dim == 'height':
        crop_size = int(h * crop_percent / 2)
        volume = volume[:, crop_size:h - crop_size, :]
    elif crop_dim == 'width':
        crop_size = int(w * crop_percent / 2)
        volume = volume[:, :, crop_size:w - crop_size]
    elif crop_dim == 'both':
        crop_size_h = int(h * crop_percent / 2)
        crop_size_w = int(w * crop_percent / 2)
        volume = volume[:, crop_size_h:h - crop_size_h, crop_size_w:w - crop_size_w]
    else:
        raise ValueError('Crop dimension must be 'height', 'width', or 'both'')

    return volume


def random_erasing_3d(volume, num_cubes=5, min_size=50, max_size=75):
    '''Erases random cubic regions in a 3D volume.'''
    d, h, w = volume.shape
    np.random.seed(seed)

    for _ in range(num_cubes):
        size = np.random.randint(min_size, max_size + 1)
        x = np.random.randint(0, d - size + 1)
        y = np.random.randint(0, h - size + 1)
        z = np.random.randint(0, w - size + 1)
        volume[x:x+size, y:y+size, z:z+size] = 0

    return volume


# Custom 3D Scaling (Height, Width, or Both)
def random_scale_3d(volume, scale_factor=0.3, axis='height'):
    '''Scales a 3D volume along height, width, or both.'''
    d, h, w = volume.shape

    if axis == 'height':
        zoom_factors = (1, scale_factor, 1)
    elif axis == 'width':
        zoom_factors = (1, 1, scale_factor)
    elif axis == 'both':
        zoom_factors = (1, scale_factor, scale_factor)
    else:
        raise ValueError('Axis must be 'height', 'width', or 'both'')

    return scipy.ndimage.zoom(volume, zoom=zoom_factors, order=1)


# Zoom with Fixed Output Dimensions
def zoom_fixed_3d(volume, zoom_factor=1.25):
    '''Zooms in or out while keeping the original dimensions.'''
    d, h, w = volume.shape
    zoomed = scipy.ndimage.zoom(volume, zoom=(1, zoom_factor, zoom_factor), order=1)
    new_h, new_w = zoomed.shape[1:]
    crop_h, crop_w = (new_h - h) // 2, (new_w - w) // 2
    return zoomed[:, crop_h:crop_h + h, crop_w:crop_w + w]


# Apply augmentations
def augment_volume(volume, config, dtype):
    '''Apply selected augmentations to a 3D volume.'''
    if config.get('mirror', False):
        volume = mirror_3d(volume, mode=config.get('mirror_mode', 'horizontal'))
    if config.get('rotate', False):
        volume = rotate_90_3d(volume, num_rot=config.get('num_rot', 1))
    if config.get('shear_crop', False):
        volume = shear_crop_3d(volume, crop_percent=0.3, crop_dim=config.get('shear_crop_dim', 'height'))
    if config.get('random_erasing', False):
        volume = random_erasing_3d(volume)
    if config.get('scale_3d', False):
        volume = random_scale_3d(volume, scale_factor=0.3, axis=config.get('scale_axis', 'height'))
    if config.get('zoom_fixed', False):
        volume = zoom_fixed_3d(volume, zoom_factor=config.get('zoom_factor', 1.25))

    return np.array(volume, dtype=dtype)


# Process and save augmented dataset
def process_and_save_tiff(input_dir, output_dir, config, dtype):
    '''Loads a TIFF stack from a directory, applies augmentations, and saves it back as individual TIFFs.'''
    volume = load_tiff_stack_from_dir(input_dir)
    augmented_volume = augment_volume(volume, config, dtype)
    save_tiff_stack_to_dir(augmented_volume, output_dir)

In [ ]:
# Main execution
if __name__ == '__main__':
    # Load dataset (NumPy-based)
    tablet_path = r'd:\Darren\Files\database\tablet_dataset\grayscale\tiff\2_Tablet'

    # Choose augmentations
    config = {
        'mirror': False, # Apply left-right and top-bottom mirroring
        'mirror_mode': 'horizontal' # Axis for mirroring
        'rotate': True, # Apply 90-degree rotation
        'num_rot': 1, # Number of 90 degree rotations to perform
        'shear_crop': False, # Apply shear cropping
        'crop_factor': 0.3, # Factor to crop the axis/axes by
        'crop_dim': 'both', # Dimension to crop along        
        'random_erasing': False, # Apply random cube erasing
        'num_cubes': 100, # Number of cubes to erase
        'scale_3d': False, # Apply random 3D scaling
        'scale_factor': 1.25, # Factor to sacle the axis/axes by
        'scale_axis': 'width', # Axis to scale along
        'zoom_fixed_3d': False, # Zoom while maintaining dimensions of image
        'zoom_factor': 1.25 # Factor to zoom in by
    }

    # Save augmented dataset
    output_path = tablet_path + '_augmented'

    # Apply augmentations with tqdm progress tracking
    augmented_tablet = process_and_save_tiff(tablet_path, output_path, config, dtype=np.uint16)